In [2]:
import csv
import pdb
import numpy as np
import random
random.seed(2000)
import ast
import os
import collections
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import cm
from matplotlib.colors import LightSource
import scipy.stats

In [3]:
os.getcwd()

'/Users/subhrasishchakraborty/Desktop'

In [4]:
os.chdir('/Users/subhrasishchakraborty/Desktop/lumbar-conformal-main/scripts')
import conformal_demo

In [5]:
num_trials=10
#alphas = np.array([0.01,0.05,0.1,0.15,0.2])
alphas = np.arange(0.01,0.21,0.01)
cal_percent = 0.35

In [7]:
df = pd.read_csv('/Users/subhrasishchakraborty/Desktop/mydata.csv')

In [8]:
df.head()

,Unnamed: 0,X1,X2,X3,cal_labels
0,3,0.000959,0.023537,0.975504,3
1,5,0.000187,0.013703,0.986111,3
2,6,0.001539,0.027498,0.970963,3
3,7,0.002468,0.032093,0.965439,3
4,8,0.109023,0.204537,0.686440,3


In [9]:
def class_conditional_eval(sets, labels, classes=['mod', 'mil', 'sev']):
    res = collections.defaultdict()
    for i, v in enumerate(classes):
        class_cond_sets = sets[labels == i]
        n = class_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (class_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
            'size': class_cond_sets.sum(1).mean(),
        }
    return res

def size_conditional_eval(sets, labels, sizes=[1, 2, 3]):
    res = collections.defaultdict()
    for i, v in enumerate(sizes):
        size_cond_sets = sets[sets.sum(1) == v]
        n = size_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (size_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
        }
    return res

In [10]:
alpha_aps_overall_coverage = collections.defaultdict(list)
alpha_aps_overall_size = collections.defaultdict(list)

alpha_aps_class_cond_count = collections.defaultdict(list)
alpha_aps_class_cond_coverage = collections.defaultdict(list)
alpha_aps_class_cond_size = collections.defaultdict(list)
alpha_aps_size_cond_count = collections.defaultdict(list)
alpha_aps_size_cond_coverage = collections.defaultdict(list)

In [12]:
for alpha in alphas:
    
    aps_overall_coverage = []
    aps_overall_size = []

    aps_class_cond_count = collections.defaultdict(list)
    aps_class_cond_coverage = collections.defaultdict(list)
    aps_class_cond_size = collections.defaultdict(list)
    aps_size_cond_count = collections.defaultdict(list)
    aps_size_cond_coverage = collections.defaultdict(list)
    aps_size_cond_size = collections.defaultdict(list)
    for trial in range(num_trials):
        
        df_cal = df.sample(frac=cal_percent)
        df_val = df.drop(df_cal.index)
        df_cal = df_cal.values
        df_val = df_val.values
        cal_scores = df_cal[:,1:4]
        val_scores = df_val[:,1:4]
        cal_labels = df_cal[:,4]-1
        val_labels = df_val[:,4]-1
        cal_labels = cal_labels.astype('int32')
        val_labels = val_labels.astype('int32')
        
        
            
        # Ordinal APS
        aps_qhat = conformal_demo.get_qhat_ordinal_aps(
            conformal_demo.ordinal_aps_prediction,
            np.copy(cal_scores),
            np.copy(cal_labels),
            alpha
        )
        aps_pred_sets = conformal_demo.ordinal_aps_prediction(np.copy(val_scores), aps_qhat)
        aps_overall_coverage.append(aps_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
        aps_overall_size.append(aps_pred_sets.sum(1).mean())
        for _class, cond_res in class_conditional_eval(aps_pred_sets, val_labels).items():
            aps_class_cond_count[_class].append(cond_res['count'])
            aps_class_cond_coverage[_class].append(cond_res['coverage'])
            aps_class_cond_size[_class].append(cond_res['size'])
        for _size, cond_res in size_conditional_eval(aps_pred_sets, val_labels).items():
            aps_size_cond_count[_size].append(cond_res['count'])
            aps_size_cond_coverage[_size].append(cond_res['coverage'])

    
    alpha_aps_overall_coverage[alpha] = aps_overall_coverage
    alpha_aps_overall_size[alpha] = aps_overall_size
        
    alpha_aps_class_cond_count[alpha] = aps_class_cond_count
    alpha_aps_class_cond_coverage[alpha] = aps_class_cond_coverage
    alpha_aps_class_cond_size[alpha] = aps_class_cond_size
    alpha_aps_size_cond_count[alpha] = aps_size_cond_count
    alpha_aps_size_cond_coverage[alpha] = aps_size_cond_coverage

/Users/subhrasishchakraborty/Desktop/lumbar-conformal-main/scripts/conformal_demo.py:120: RuntimeWarning: Mean of empty slice.
  covered[val_labels == j].mean() for j in range(np.unique(val_labels).max() + 1)
/Users/subhrasishchakraborty/miniforge3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
aps_agg_coverage = np.array(list(alpha_aps_overall_coverage.values()))
aps_agg_size = np.array(list(alpha_aps_overall_size.values()))

In [14]:
aps_agg_cvg=[]
for x in aps_agg_coverage:
    aps_agg_cvg.append(x.mean())

In [24]:
aps_agg_set_size=[]
for x in aps_agg_size:
    aps_agg_set_size.append(x.mean())

In [17]:
aps_sev_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['sev'] for alpha in alphas])
aps_mod_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mod'] for alpha in alphas])
aps_mil_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mil'] for alpha in alphas])

In [23]:
aps_mil_cvg = []
for x in aps_mil_coverage:
    aps_mil_cvg.append(x.mean())

In [22]:
aps_mod_cvg = []
for x in aps_mod_coverage:
    aps_mod_cvg.append(x.mean())

In [21]:
aps_sev_cvg = []
for x in aps_sev_coverage:
    aps_sev_cvg.append(x.mean())